In [2]:
import torch
from transformers import pipeline
from PIL import Image, ImageFile
import numpy as np
import os
from tqdm import tqdm

# PIL 라이브러리의 이미지 로드 중 오류를 무시하도록 설정
ImageFile.LOAD_TRUNCATED_IMAGES = True

def remove_background(image_path, device, pipe):
    try:
        pillow_mask = pipe(image_path, return_mask=True)
        pillow_image = pipe(image_path)
    except IOError as e:
        print(f"Error processing {image_path}: {e}")
        return None  # 오류 발생 시 None 반환

    np_image = np.array(pillow_image)
    np_image[np_image == 0] = 255

    modified_image = Image.fromarray(np_image)
    if modified_image.mode == 'RGBA':
        modified_image = modified_image.convert('RGB')

    return modified_image

def process_images(input_folder, output_folder, error_log_path):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    pipe = pipeline("image-segmentation", model="briaai/RMBG-1.4", device=device, trust_remote_code=True)
    errors = []

    for filename in tqdm(os.listdir(input_folder)):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(input_folder, filename)
            result_image = remove_background(image_path, device, pipe)
            if result_image is None:
                errors.append(filename)
            else:
                output_path = os.path.join(output_folder, filename)
                result_image.save(output_path)

    # 오류가 발생한 파일 이름을 로그 파일에 기록
    if errors:
        with open(error_log_path, 'w') as f:
            for item in errors:
                f.write("%s\n" % item)

# 경로 설정
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "2"

input_folder = '/home/mmc/disk2/duck/cap/data/snack/val/images'
output_folder = '/home/mmc/disk2/duck/cap/data/snack_RMBG/val/images'
error_log_path = 'error_snack_val.log'

# 이미지 처리 실행
process_images(input_folder, output_folder, error_log_path)


100%|██████████| 12386/12386 [2:28:55<00:00,  1.39it/s] 


In [1]:
import os
print(len(os.listdir('/home/mmc/disk2/duck/cap/data/snack/val/images')), len(os.listdir('/home/mmc/disk2/duck/cap/data/snack/val/images_224')))



12386 12386
